# Regression Predict Student Solution

© Explore Data Science Academy

---
### Honour Code

We {**TEAM RR15**}, confirm - by submitting this document - that the solutions in this notebook are a result of my own work and that I abide by the [EDSA honour code](https://drive.google.com/file/d/1QDCjGZJ8-FmJE3bZdIQNwnJyQKPhHZBn/view?usp=sharing).

Non-compliance with the honour code constitutes a material breach of contract.

### Predict Overview: Spain Electricity Shortfall Challenge

The government of Spain is considering an expansion of it's renewable energy resource infrastructure investments. As such, they require information on the trends and patterns of the countries renewable sources and fossil fuel energy generation. Your company has been awarded the contract to:

- 1. analyse the supplied data;
- 2. identify potential errors in the data and clean the existing data set;
- 3. determine if additional features can be added to enrich the data set;
- 4. build a model that is capable of forecasting the three hourly demand shortfalls;
- 5. evaluate the accuracy of the best machine learning model;
- 6. determine what features were most important in the model’s prediction decision, and
- 7. explain the inner working of the model to a non-technical audience.

Formally the problem statement was given to you, the senior data scientist, by your manager via email reads as follow:

> In this project you are tasked to model the shortfall between the energy generated by means of fossil fuels and various renewable sources - for the country of Spain. The daily shortfall, which will be referred to as the target variable, will be modelled as a function of various city-specific weather features such as `pressure`, `wind speed`, `humidity`, etc. As with all data science projects, the provided features are rarely adequate predictors of the target variable. As such, you are required to perform feature engineering to ensure that you will be able to accurately model Spain's three hourly shortfalls.
 
On top of this, she has provided you with a starter notebook containing vague explanations of what the main outcomes are. 

<a id="cont"></a>

## Table of Contents

<a href=#one>1. Importing Packages</a>

<a href=#two>2. Loading Data</a>

<a href=#three>3. Exploratory Data Analysis (EDA)</a>

<a href=#four>4. Data Engineering</a>

<a href=#five>5. Modeling</a>

<a href=#six>6. Model Performance</a>

<a href=#seven>7. Model Explanations</a>

 <a id="one"></a>
## 1. Importing Packages
<a href=#cont>Back to Table of Contents</a>

---
    
| ⚡ Description: Importing Packages ⚡ |
| :--------------------------- |
| In this section you are required to import, and briefly discuss, the libraries that will be used throughout your analysis and modelling. |

---

In [ ]:
# Libraries for data loading, data manipulation and data visulisation
import seaborn as sns 
import matplotlib.pyplot as plt
import plotly.express as px 
import plotly.graph_objects as go
import plotly.io as pio
pio.templates
import matplotlib.style as style
from plotly.subplots import make_subplots
from scipy import stats
import pickle
%matplotlib inline

# Libraries for data preparation and model building
import numpy as np
import pandas as pd

# Setting global constants to ensure notebook results are reproducible
PARAMETER_CONSTANT = 42

<a id="two"></a>
## 2. Loading the Data
<a class="anchor" id="1.1"></a>
<a href=#cont>Back to Table of Contents</a>

---
    
| ⚡ Description: Loading the data ⚡ |
| :--------------------------- |
| In this section you are required to load the data from the `df_train` file into a DataFrame. |

---

In [ ]:
df_train = pd.read_csv('df_train.csv')
df_test = pd.read_csv('df_test.csv')

<a id="three"></a>
## 3. Exploratory Data Analysis (EDA)
<a class="anchor" id="1.1"></a>
<a href=#cont>Back to Table of Contents</a>

---
    
| ⚡ Description: Exploratory data analysis ⚡ |
| :--------------------------- |
| In this section, you are required to perform an in-depth analysis of all the variables in the DataFrame. |

---


In [ ]:
# look at data statistics
df_train.head()

In [ ]:
df_test.head()

In [ ]:
df_train.shape

In [ ]:
df_train.describe()

In [ ]:
df_train.info()

In [ ]:
#Check for null values
df_train.isnull().sum()

In [ ]:
# plot relevant feature interactions
sns.boxplot(df_train['Valencia_pressure'])

In [ ]:
# evaluate correlation

In [ ]:
# have a look at feature distributions

<a id="four"></a>
## 4. Data Engineering
<a class="anchor" id="1.1"></a>
<a href=#cont>Back to Table of Contents</a>

---
    
| ⚡ Description: Data engineering ⚡ |
| :--------------------------- |
| In this section you are required to: clean the dataset, and possibly create new features - as identified in the EDA phase. |

---

In [ ]:
#concatinate test and train for easy data processing

df=pd.concat([df_train,df_test])

df.head()

In [ ]:
# remove missing values/ features
df_new=df

#impute null values in Valencia pressure with mode
print('mode')
print(df_new['Valencia_pressure'].mode())
print('median')
print(df_new['Valencia_pressure'].median())
print('mean')
print(df_new['Valencia_pressure'].mean())



In [ ]:
# create new features
df_new['Valencia_pressure'] = df_new['Valencia_pressure'].fillna(df_new['Valencia_pressure'].mode()[0])
#convert objects into numeric form


In [ ]:
df_new.isnull().sum()

In [ ]:
#Weconverted Date column from object to datetime
df_new['time'] = pd.to_datetime(df_new['time'])

#We created new columns Year, Month, Day from the Date column
df_new['Day'] = df_new['time'].dt.day
df_new['Month'] = df_new['time'].dt.month
df_new['Year'] = df_new['time'].dt.year
df_new['hour'] = df_new['time'].dt.hour
#Then we drop the Date column since we already broke it down to three columns(Year,Month and Day)
df_new.drop(['time'], inplace=True, axis=1)
 

In [ ]:
df_new.info()

In [ ]:

df_new['Valencia_wind_deg'] = df_new['Valencia_wind_deg'].str.extract('(\d+)')


In [ ]:
df_new['Valencia_wind_deg'] 

In [ ]:
df_new['Valencia_wind_deg'] = pd.to_numeric(df_new['Valencia_wind_deg'])

In [ ]:
df_new['Seville_pressure'] = df_new['Seville_pressure'].str.extract('(\d+)')


In [ ]:
df_new['Seville_pressure'] 

In [ ]:
df_new['Seville_pressure'] = pd.to_numeric(df_new['Seville_pressure'])

In [ ]:
df_new.info()

In [ ]:
# engineer existing features

In [ ]:
df_new=df_new.drop(['Unnamed: 0'],axis=1)

<a id="five"></a>
## 5. Modelling
<a class="anchor" id="1.1"></a>
<a href=#cont>Back to Table of Contents</a>

---
    
| ⚡ Description: Modelling ⚡ |
| :--------------------------- |
| In this section, you are required to create one or more regression models that are able to accurately predict the thee hour load shortfall. |

---

In [ ]:
# split data

y = df_new[:len(df_train)][['load_shortfall_3h']]
x = df_new[:len(df_train)].drop('load_shortfall_3h',axis=1)

x_test = df_new[len(df_train):].drop('load_shortfall_3h',axis=1)
y_train=df_new[:len(df_train)][['load_shortfall_3h']]

In [ ]:
x.head()

In [ ]:
y_train.head()

In [ ]:
df_ans=y

print('mode')
print(df_ans['load_shortfall_3h'].mode())
print('median')
print(df_ans['load_shortfall_3h'].median())
print('mean')
print(df_ans['load_shortfall_3h'].mean())

In [ ]:
x_test.head()

In [ ]:
# create targets and features dataset
from sklearn.model_selection import train_test_split

# x is the independent variable(s) and y is the dependent variable 'load_shortfall_3h'
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=50)

In [ ]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()

In [ ]:
# create one or more ML models

lr.fit(x_train,y_train)
preds = lr.predict(x_test)

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error

In [ ]:
# evaluate one or more ML models

def root_mse(y_test,y_predict):
    return np.sqrt(mean_squared_error(y_test,y_predict))

In [ ]:
root_mse(y_test, preds)

In [ ]:
r2_score(y_test, preds)

In [ ]:
mean_absolute_error(y_test, preds)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error


# define the model
rf_model = RandomForestRegressor(n_estimators=200, random_state=50)

# train the model
rf_model.fit(x_train, y_train)

# make predictions on the test data
y_pred = rf_model.predict(x_test)

# evaluate the model
r2 = rf_model.score(x_test, y_test)
print('RMSE:', root_mse(y_test,y_pred))
print('R2 score:', r2)


In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error

# define the model
gb_model = GradientBoostingRegressor(n_estimators=100, max_depth=7, random_state=50)

# train the model
gb_model.fit(x_train, y_train)

# make predictions on the test data
y_pred = gb_model.predict(x_test)

# evaluate the model
r2 = gb_model.score(x_test, y_test)
print('RMSE:', root_mse(y_test,y_pred))
print('R2 score:', r2)


<a id="six"></a>
## 6. Model Performance
<a class="anchor" id="1.1"></a>
<a href=#cont>Back to Table of Contents</a>

---
    
| ⚡ Description: Model performance ⚡ |
| :--------------------------- |
| In this section you are required to compare the relative performance of the various trained ML models on a holdout dataset and comment on what model is the best and why. |

---

In [ ]:
# Compare model performance
compare={'r2 score for linear regression':r2_score(y_test,lr_preds),
        'r2 score for Random Forest Regression':r2_score(y_test,y_pred),
        'r2 score for Gradient boost regression':r2_score(y_test,gb_y_pred)}
compare

In [ ]:
# Choose best model and motivate why it is the best choice
'''
Random Forest Regression is the best model because it has the highest r2 score
'''

<a id="seven"></a>
## 7. Model Explanations
<a class="anchor" id="1.1"></a>
<a href=#cont>Back to Table of Contents</a>

---
    
| ⚡ Description: Model explanation ⚡ |
| :--------------------------- |
| In this section, you are required to discuss how the best performing model works in a simple way so that both technical and non-technical stakeholders can grasp the intuition behind the model's inner workings. |

---

In [ ]:
# discuss chosen methods logic